<a href="https://colab.research.google.com/github/DASIREDDYCHAITANYA/Practice_hub-GEN-AI/blob/main/LLM_RAG(CHATBOT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install pypdf
!pip install langchain
!pip install faiss-cpu
!pip install langchain_huggingface


In [ ]:
import getpass
import os

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your token: ")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
loader=PyPDFLoader("/content/attention.pdf")
data=loader.load()
data

In [ ]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)
data=splitter.split_documents(data)
data[:5]

In [20]:
vector=FAISS.from_documents(data,embeddings)

In [23]:
query = "what is Recurrent neural networks?"
docs = vector.similarity_search(query)
print(docs[0].page_content)

1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and


In [36]:
retriver=vector.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [37]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "google/flan-t5-large"  # 1024 token limit
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=700, truncation=True)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cuda:0


In [38]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [39]:
from langchain.prompts import PromptTemplate

prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [40]:
from langchain.chains import RetrievalQA

retrievalQA=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retriver,chain_type_kwargs={"prompt":prompt})

In [41]:
query = "what is Recurrent neural networks?"
result=retrievalQA.run(query)
print(result)

state of the art approaches in sequence modeling and Recognition
